In [ ]:
# %load /home/justin/invest_round.sh
#! /bin/bash
# echo $PATH 2>&1 | mail -s pre_source ruwenus@gmail.com
source justin_tinkering/justin_tinker/bin/activate
export PATH="/home/justin/bin:/home/justin/.local/bin:/usr/local/cuda-8.0/bin:/home/justin/anaconda3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin$ "
#export PATH="${PATH}:/home/justin/justin_tinkering/data_science/"
# echo $PATH 2>&1 | mail -s post_source ruwenus@gmail.com
#export PYTHONPATH="${PYTHONPATH}:/home/justin/anaconda3/bin/python/"
#export PYTHONPATH="${PYTHONPATH}:/home/justin/anaconda3/lib/python3.6/site-packages/"
export PYTHONPATH="${PYTHONPATH}:/home/justin/justin_tinkering/data_science/"
# python -c "import sys; print '\n'.join(sys.path)" 2>&1 | mail -s python_post_SOURCE ruwenus@gmail.com

# virtualenv is now active, which means your PATH has been modified.
# Don't try to run python from /usr/bin/python, just run "python" and
# let the PATH figure out which version to run (based on what your
# virtualenv has configured).

python ~/justin_tinkering/data_science/lendingclub/investing/invest_script.py 2>&1 | mail -s `date +"%Y/%m/%d-%H:%M:%S"`_investment_round ruwenus@gmail.com
#sleep 1s
#python ~/justin_tinkering/data_science/lendingclub/counting_loans_batches/count_loans.py 2>&1 | mail -s `date +"%Y/%m/%d-%H:%M:%S"`loan_count_after_invest_round ruwenus@gmail.com

sleep 5s

python ~/justin_tinkering/data_science/lendingclub/investing/invest_script_instant.py 2>&1 | mail -s `date +"%Y/%m/%d-%H:%M:%S"`_investment_round ruwenus@gmail.com
#sleep 1s
#python ~/justin_tinkering/data_science/lendingclub/counting_loans_batches/count_loans.py 2>&1 | mail -s `date +"%Y/%m/%d-%H:%M:%S"`loan_count_after_invest_round ruwenus@gmail.com


In [9]:
!export PATH="/home/justin/bin:/home/justin/.local/bin:/usr/local/cuda-8.0/bin:/home/justin/anaconda3/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin$ "

In [10]:
!export PYTHONPATH="${PYTHONPATH}:/home/justin/justin_tinkering/data_science/"

In [11]:
!python invest_script_instant.py

From DL Server, instant invest
Traceback (most recent call last):
  File "invest_script_instant.py", line 6, in <module>
    import lendingclub.account_info as acc_info
ModuleNotFoundError: No module named 'lendingclub'


In [ ]:
# %load /home/justin/ethermine_alert/pa_ethermine_alert.py
#!/usr/bin/env python
#
# Written by /u/ExploitZero. Constructive feedback is always welcome. Feel free to modify this script for your own needs.
#
# Purpose:
#
#    Monitor Ethermine to determine whether your rigs are active and performing as expected.
#
# Usage:
#
#    Run this script from a Linux system that is NOT one of your rigs.
#
#    1) Create a new Gmail account with a username and password you do not use for other websites.
#    2) Copy this script into your preferred directory. Ensure that it is executable by issuing the 'chmod a+x ethermine_alert.py' command.
#    3) Use a text editor (e.g., nano) to replace all items in USER DEFINED VARIABLES section with your rig's wallet address, your new gmail credentials, and the email to SMS addresses you'd like to receive the alerts.
#    4) Issue the 'crontab -e' command and append '*/5 * * * * python /full/path/to/ethermine_alert.py' to the file (for a check every 5 minutes).
#
#    If you would like to run a test to make sure the script is working, modify the 'my_total_hashrate_threshold' variable to a number absurdly high for your workers, then issue the './ethermine_alert.py' command
#
# Feeling generous?
#
#     ETH:0x09439AD892a676a814aEA49aF2e0e8ee3106F11B
#     BTC:14y78wmeJJhgZR9w1q468zvbmqFaWGKsku
#

##########################
# USER DEFINED VARIABLES #
##########################

my_wallet_address = '0x176b4936219893955fFc3dc2f5E28ecA2354751e'
my_gmail_account = 'justin.hsi.mine.alert@gmail.com'
my_gmail_password = 'abcdef8868'
my_recipients = ['ruwenus@gmail.com','zurenh@yahoo.com', 'jzhsi@yahoo.com']
my_total_hashrate_threshold = 180
my_worker_hashrate_threshold = 180

########
# Main #
########

import urllib.request as urllib2
import json
import smtplib

issue = False
message = 'There may be an issue with one of your rigs.\n\n'

json_output = json.load(urllib2.urlopen(urllib2.Request('https://ethermine.org/api/miner_new/'+ my_wallet_address, headers={'User-Agent' : 'Magic Browser'})))
for majorkey, subdict in json_output.items():
    if majorkey == 'workers':
        worker_names = subdict
    if majorkey == 'hashRate':
        total_hashrate = subdict

if float(total_hashrate.split(" ")[0]) < my_total_hashrate_threshold:
    issue = True
    message += 'Current effective hashrate is %s, which is below the threshold set at %s MH/s.\n\n' % (total_hashrate, my_total_hashrate_threshold)

for key, subdict in worker_names.items():
    worker_name = subdict
    for key, value in worker_name.items():
        if key == 'worker':
            message += "%s:%s\n" % (key,value)
        if key == 'hashrate':
            message += "    %s:%s\n" % (key,value)
            if float(value.split(" ")[0]) < my_worker_hashrate_threshold:
                issue = True
                message += 'Hashrate is below the threshold set at %s MH/s.\n\n' % (my_worker_hashrate_threshold)

def send_emails():
    subject = 'Rig Issue'
    smtpserver = smtplib.SMTP('smtp.gmail.com',587)
    smtpserver.ehlo()
    smtpserver.starttls()
    smtpserver.ehlo()
    smtpserver.login(my_gmail_account, my_gmail_password)
    msg = """From: %s\nTo: %s\nSubject: %s\n\n%s""" % (my_gmail_account, my_recipients, subject, message)
    smtpserver.sendmail(my_gmail_account, my_recipients, msg)
    smtpserver.close()

if issue == True:
    send_emails()
else:
    pass
